In [ ]:
from flask import Flask, render_template
import pandas as pd
from datetime import datetime

app = Flask(__name__)

data = pd.read_csv("2008.csv")

@app.route('/')
def index():
    current_time = datetime.now().time().strftime("%I:%M %p")

    itinerary = data.loc[(data['Month'] == datetime.now().month) & (data['DayofMonth'] == datetime.now().day), ['DepTime', 'ArrTime', 'FlightNum', 'Origin', 'Dest']]
    itinerary = itinerary.dropna()

    delays = data[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]
    cancellations = data[['Cancelled']]

    def parse_time(time_str):
        if time_str is None or pd.isna(time_str):
            return None
        try:
            time_obj = datetime.strptime(str(time_str).split('.')[0], "%H%M").time()
            return time_obj
        except ValueError:
            return None

    def has_delay(row):
        for column in delays:
            if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] >= 15:
                return True
        return False

    def cancelled(row):
        for column in cancellations:
            if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] != 0:
                return True
        return False

    def arrivals(airport):
        current_time = datetime.now().time()
        filtered_rows = [row for row in data.itertuples() if row.Origin == airport]
        results = []
        for row in filtered_rows:
            dep_time = parse_time(row.DepTime)
            arr_time = parse_time(row.ArrTime)
            if dep_time is not None and arr_time is not None and row.Dest == airport and dep_time <= current_time <= arr_time:
                results.append(row)
        return results

    def departures(airport):
        departures_list = []
        for _, row in itinerary.iterrows():
            dep_time = parse_time(row['DepTime'])
            if dep_time is not None and row['Origin'] == airport and datetime.now().time() <= dep_time:
                status = "Delayed" if has_delay(row) else "Cancelled" if cancelled(row) else "On Time"
                departures_list.append((row['FlightNum'], row['Origin'], row['Dest'], status, dep_time))
        return departures_list

    return render_template('index.html', current_time=current_time, arrivals=arrivals('TPA'), departures=departures('TPA'))

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jun/2023 19:27:38] "GET / HTTP/1.1" 200 -
